In [23]:
from dotenv import load_dotenv
import os

load_dotenv()

YOUR_APP_CLIENT_ID = os.getenv("YOUR_APP_CLIENT_ID")
YOUR_APP_CLIENT_SECRET = os.getenv("YOUR_APP_CLIENT_SECRET")

In [24]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=YOUR_APP_CLIENT_ID,
    client_secret=YOUR_APP_CLIENT_SECRET,
    redirect_uri="http://127.0.0.1:8000/callback",
    scope="user-library-read,user-read-playback-state,user-read-playback-position,user-top-read,user-read-recently-played,user-follow-read",
))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 HalaCG  –  Big Buff Woman
1 Emilia  –  noviogangsta <3
2 SPIIRAL  –  THE ONE
3 Måneskin  –  Morirò da Re


In [9]:
sp.current_user()

{'display_name': 'Meleagrista',
 'external_urls': {'spotify': 'https://open.spotify.com/user/xmartinricox'},
 'followers': {'href': None, 'total': 4},
 'href': 'https://api.spotify.com/v1/users/xmartinricox',
 'id': 'xmartinricox',
 'images': [{'height': 300,
   'url': 'https://i.scdn.co/image/ab6775700000ee85903fbc62de0625623938bc2e',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67757000003b82903fbc62de0625623938bc2e',
   'width': 64}],
 'type': 'user',
 'uri': 'spotify:user:xmartinricox'}

In [12]:
sp.current_user_recently_played(limit=3)

{'items': [{'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5uzPIJDzWAujemRDKiJMRj'},
       'href': 'https://api.spotify.com/v1/artists/5uzPIJDzWAujemRDKiJMRj',
       'id': '5uzPIJDzWAujemRDKiJMRj',
       'name': 'DEATHPIXIE',
       'type': 'artist',
       'uri': 'spotify:artist:5uzPIJDzWAujemRDKiJMRj'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/1GzMtNyLiP1oM7yW0DyRhP'},
       'href': 'https://api.spotify.com/v1/artists/1GzMtNyLiP1oM7yW0DyRhP',
       'id': '1GzMtNyLiP1oM7yW0DyRhP',
       'name': 'CYPARISS',
       'type': 'artist',
       'uri': 'spotify:artist:1GzMtNyLiP1oM7yW0DyRhP'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT

In [13]:
sp.current_user_top_artists(limit=3)

{'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/45dkTj5sMRSjrmBSBeiHym'},
   'followers': {'href': None, 'total': 7780955},
   'genres': [],
   'href': 'https://api.spotify.com/v1/artists/45dkTj5sMRSjrmBSBeiHym',
   'id': '45dkTj5sMRSjrmBSBeiHym',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb590259e859082dc3b661b8ac',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab67616100005174590259e859082dc3b661b8ac',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f178590259e859082dc3b661b8ac',
     'width': 160}],
   'name': 'Tate McRae',
   'popularity': 90,
   'type': 'artist',
   'uri': 'spotify:artist:45dkTj5sMRSjrmBSBeiHym'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/5qvrAlKjc0Sqrqz6v2wBGk'},
   'followers': {'href': None, 'total': 1320080},
   'genres': [],
   'href': 'https://api.spotify.com/v1/artists/5qvrAlKjc0Sqrqz6v2wBGk',
   'id

In [14]:
sp.current_user_top_tracks(limit=3)

{'items': [{'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/09cKncAQn28NqTUORLMwSR'},
      'href': 'https://api.spotify.com/v1/artists/09cKncAQn28NqTUORLMwSR',
      'id': '09cKncAQn28NqTUORLMwSR',
      'name': 'Ariis',
      'type': 'artist',
      'uri': 'spotify:artist:09cKncAQn28NqTUORLMwSR'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
     'A

In [19]:
TRACK_ID="11dFghVXANMlKmJXsNCbNl"

In [17]:
track = sp.track(TRACK_ID)
track

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6sFIWsNpZYqfjUpaCgueju'},
    'href': 'https://api.spotify.com/v1/artists/6sFIWsNpZYqfjUpaCgueju',
    'id': '6sFIWsNpZYqfjUpaCgueju',
    'name': 'Carly Rae Jepsen',
    'type': 'artist',
    'uri': 'spotify:artist:6sFIWsNpZYqfjUpaCgueju'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/0tGPJ0bkWOUmH7MEOR77qc'},
  'href': 'https://api.spotify.com/v1/albums/0tGPJ0bkWOUmH7MEOR77qc',
  'id': '0tGPJ0bkWOUmH7MEOR77qc',
  'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b2737359994525d219f64872d3b1',
    'width': 640,
    'height': 640},
   {'url': 'https://i.scdn.co/image/ab67616d00001e027359994525d219f64872d3b1',
    'width': 300,
    'height': 300},
   {'url': 'https://i.scdn.co/image/ab67616d000048517359994525d219f64872d3b1',
    'width': 64,
    'height': 64}],
  'name': 'Cut To The Feeling',
  'release_date': '2017-05-26',
 

In [ ]:
# feats = sp.audio_features([TRACK_ID])
# feats

In [ ]:
# analysis = sp.audio_analysis(TRACK_ID)
# analysis